#### Жаль что нет логов кликов по выданным документам. И хронологии с привязкой к пользователю.
#### Задача напоминает нахождение похожих предложений. надо просто снизить планку похожести для увеличения размера групп запросов. 
####    Известно, что если запросы состоят из похожих слов, то и запросы похожи. Для сравнения смыслов слов исползуется Word2Vec. Либо можно задать в цифровом варианте сразу смысл целых запросов. Это прямая задача для Doc2Vec.
####  Далее надо определиться с колличеством желаемых групп. Это скорее зависит от цели кластеризации.
##### сначала попробуем просто найти одинаковые запросы независимо от колличества групп:

- по-старинке выберем одинаковые запросы по вхождению слов

- простое сравнение Doc2Vec

- Для разных типов обработки запросов можно сгруппировать их всего на 4 группы:   ['купить','смотреть/читать','инфо','другое']. для этого я задала точное колличество групп для группировки k-means с вручную выбранными центрами 
    
- можно попробовать найти реальное количество групп и дальше придумать их обработку. Я попыталсь сделать это с помощью elbow-метода для k-means

In [2]:
import re
import requests
import gensim
from gensim.models import Phrases, Doc2Vec, Word2Vec
import pymorphy2
import pandas as pd
from io import StringIO
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn import metrics

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def stem(row):
    ar = row.lower().split()
    for i in range(len(ar)):    
        ar[i] = morph.parse(ar[i])[0].normal_form
    return ar

morph = pymorphy2.MorphAnalyzer()
link = 'https://export.yandex.ru/last/last20x.xml'

In [82]:
#сбор даных с яндекса. после ~140тысяч уникальные длинные запросы стали добавляться слишком медленно.
d = pd.DataFrame()
while len(d)<10000:
    f = requests.get(link).text
    f = cleanhtml(f)
    tex = StringIO(f)
    d1= pd.read_csv(tex, sep='\n')
    d1.columns = ['text']
    ind = d1[d1['text'].str.contains(' [a-zA-Z] |&')].index
    d1.drop(ind,inplace=True)
    d1['text'] = d1['text'].apply(lambda x: re.sub('[123456.,7890-=+_)(*^%$#@!~`|\/)?><":]| {2,11}', '', x))
    d1['spac'] = d1['text'].apply(lambda x: x.count(' '))
    #d1 = d1[d1['spac']>1] #для увеличения точности обучаемых моделей брала запросы не короче 3х слов
    d = d.append(d1,ignore_index= True)
    #d.drop_duplicates('text',inplace=True) #для тренировки модели обойдемся без дубликатов

In [87]:
d['split_stem'] = d['text'].apply(stem)
ar = list(d['split_stem'])

#train phrases
bg = Phrases(ar) 
d['bgrams'] = d['split_stem'].apply(lambda x: bg[x])

#make LabeledSentences set for Doc2Vec training
d = d.reset_index()
d['tpls'] = d.apply(lambda x: (x['bgrams'],x['index']),1)
d['tpls'] = d['tpls'].apply(lambda x: gensim.models.doc2vec.LabeledSentence(x[0],[x[1]]))
sentences = list(d['tpls'])
d.drop(['tpls','spac'],1, inplace = True)

model = Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(sentences)
for epoch in range(10):
    model.train(sentences)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha

/home/zoy/anaconda3/lib/python3.5/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


#### Вот 5 рандомно выбранных строк получившейся таблицы.  Видно, что фразы в толбце 'bgrams' отработали неплохо

In [26]:
d.sample(5)

,index,text,split_stem,bgrams
4401,4401,Крайстчерч,[крайстчерч],[крайстчерч]
1402,1402,купить пляжную одежду в интернет магазине недо...,"[купить, пляжный, одежда, в, интернет, магазин...","[купить, пляжный, одежда, в, интернет_магазин,..."
573,573,как устранить скрип подошвы обуви,"[как, устранить, скрип, подошва, обувь]","[как, устранить, скрип, подошва, обувь]"
1211,1211,рапорт на ветеранский отпуск,"[рапорт, на, ветеранский, отпуск]","[рапорт, на, ветеранский, отпуск]"
205,205,форум для встречи поставщиков и покупателей сп...,"[форум, для, встреча, поставщик, и, покупатель...","[форум, для, встреча, поставщик, и, покупатель..."


####   Для точности определения по вхождениям лучше исключить стоп-слова и использовать расстояние левенштейна. 
<b> Но составление списка стоп-слов это уже отдельная задача,</b>  <i> (например слова "что, это, такое" обычно идут в стоп- слова, но тут они могут формировать отдельную группу запросов типа инфо. и "укрф что это" и "укрф читать" должны быть в разных группах)</i>,<b>   да и я уже составила словосочетания. Запросы будут сравниваться со всеми другими запросами, что подразумевает квадратичное время выполнения. Поэтому я возьму только 10000 запросов, буду сравнивать найденные словосочетания как одно слово и не брать слова короче 3х символов (это отбросит большинство предлогов). Группировать буду если у взятого для сравенния запроса больше половины слов присутствует в сравниваемом запросе.</b>


* btw: elasticsearch можно научить это делать. и он справляется за доли секунды

In [119]:
data = d.sample(10000)
related=[]
for i in data.index:
    try:
        str1 = set([x for x in data.loc[i,'bgrams'] if len(x)>2])
        for j in data.index[i+1:]:
            str2 = data.loc[j,'bgrams']
            cnt = 0
            for word in set([x for x in str1 if len(x)>2]):
                if word in str2:
                    cnt += 1
            if len(str2)>0 and cnt/len(str2) > 0.6:
                related.append((i,j))
                data.drop(j,inplace = True)
    except Exception:
        pass
related = pd.DataFrame(related)
related.groupby('query').size().sort_values(ascending = False)[:10] #размеры групп

query
785     8
783     7
2292    5
402     5
5470    4
3185    4
380     4
788     4
6138    3
267     3
dtype: int64

#### Левый столбец- номер запроса на который собраны похожие, правый- кооличество похожих на этот запрос. Посмотрим, что вошло в эти группы

In [122]:
related['related_text'] =related['related'].apply(lambda x: d.loc[x,'text'])
related[related['query']==785] #там, где related == query это взятый запрос.

,query,related,related_text
117,785,126166,фильмы смотреть онлайн
118,785,53121,лего фильм смотреть онлайн
119,785,108523,смотреть русские фильмы
120,785,785,смотреть онлайн русский фильм
121,785,121667,фильм саранча смотреть онлайн
122,785,126609,людмила гурченко фильм смотреть онлайн
123,785,89403,смотреть онлайн фильм метод серия
124,785,132198,фильм бумеранг смотреть онлайн


In [123]:
related[related['query']==783]

,query,related,related_text
128,783,23017,верни мою любовь смотреть онлайн бесплатно
129,783,120025,молодёжка сезон серия смотреть онлайн бесплатно
130,783,126674,мадагаскар смотреть онлайн бесплатно
131,783,28200,гончие смотреть онлайн бесплатно
132,783,31089,дикари смотреть онлайн бесплатно
133,783,783,смотреть онлайн бесплатно
134,783,89307,смотреть онлайн бесплатно братоны


#### Результаты действительно релевантны. как видно в основном взятый запрос входит на 100% в более длинный сравниваемый.
#### Это можно использовать в обе стороны: и если ничего не нашлось на слишком подробный запрос предложить короче. или предложить расширения для коротких.
#### С другой стороны эти же первые две группы можно слить в одну, хоть разница и не в синонимичных словах. надеюсь тут мне поможет Doc2Vec.
#### В Doc2Vec можно просто попросить показать похожие наиденные предложения с заданным трешхолдом. поэтому тут не надо проходить по сету циклом. посмотрим похожие предложения для наиденных выше групп по 783 и 785 запросам но для всего сета.

In [134]:
doc_sim = modeld.docvecs.most_similar(783, topn=15)
[(d.loc[x[0],'text'], x[1]) for x in doc_sim]

[('дикари смотреть онлайн бесплатно', 0.934154748916626),
 (' смотреть онлайн бесплатно', 0.9223512411117554),
 ('гаишникисезонсерия смотреть онлайн бесплатно', 0.8979048728942871),
 ('гаишникисезон смотреть онлайн бесплатно', 0.8970521092414856),
 ('иллюзионист смотреть онлайн', 0.8942283391952515),
 ('мобкино смотреть онлайн бесплатно', 0.893585205078125),
 ('верни мою любовь смотреть онлайн бесплатно', 0.8927480578422546),
 ('зомбиру смотреть бесплатно', 0.8901247978210449),
 ('альфаамериканцы смотреть онлайн', 0.8851928114891052),
 ('смотреть доми онлайн', 0.884375810623169),
 ('библиотекари смотреть онлайн', 0.8838937282562256),
 ('карповсезон смотреть онлайн', 0.8834539651870728),
 ('гончие смотреть онлайн бесплатно', 0.8831233978271484),
 ('сплетница смотреть онлайн', 0.8826776742935181),
 ('смешарикисериянекультурный смотреть онлайн', 0.8802814483642578)]

#### Все запросы можно считать однотипными, даже если они менее, чем на 50% совпадают по словам. у Doc2Vec определенно больше перспектив.  
#### А что есть попробовать разбить запросы на тип желаемого результата, такие как смотреть, купить, что такое..? 
####  Возьму за центры запросы состоящие из ключевых слов желаемых групп:

In [249]:
import numpy as np
init
init = np.array([modeld.docvecs[92940],modeld.docvecs[51499],modeld.docvecs[119622],modeld.docvecs[9]])
kmeans = KMeans(n_clusters=4, init=init).fit(a)
d['label'] = kmeans.labels_

/home/zoy/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [250]:
d.loc[[92940,51499,5443,1004]] #цетры съехали в одну точку( надо наверное лучше работать с центрами

,text,stem,spl,bgrams,lab,label
92940,стоит ли читать книгу гарри поттер если смотре...,стоить ли читать книга гарри поттер если смотр...,"[стоить, ли, читать, книга, гарри, поттер, есл...","[стоить_ли, читать, книга, гарри_поттер, если,...",0,0
51499,купить одежду huppa в интернет магазине,купить одежда huppa в интернет магазин,"[купить, одежда, huppa, в, интернет, магазин]","[купить, одежда, huppa, в, интернет_магазин]",2,2
5443,iga что это такое,iga что это такой,"[iga, что, это, такой]","[iga, что_это_такой]",2,2
1004,altj ∆ – lovely day,altj ∆ – lovely day,"[altj, ∆, –, lovely, day]","[altj, ∆, –, lovely, day]",2,2


#### Посмотрим какие запросы теперь ближе всего к центрам

In [308]:
for i in range(4):
    print(d.loc[modeld.docvecs.most_similar(positive=[kmeans.cluster_centers_[i]])[0][0],'text'])

какое сочинение можно написать про кухню  класс
джеки сэндлер фильмы
war of change
aragorn the king of magic land


#### ну тоже неплохо. посмотрим теперь релевантность запроссов в этих группах

In [331]:
d.groupby('label').size()

label
0    11953
1    27009
2    62724
3    32211
dtype: int64

In [335]:
d[d['label'] == 0][['text','label']].sample(5)

,text,label
127492,информатика класс скачать яндекс диск,0
24335,во сколько будет сегодня по каналу че макс,0
24274,внешняя и внутреня испания на современном этапе,0
75389,пример вин кода тойота королла америка машины,0
97085,тренажеры детские бэйби джимвелотренажер инстр...,0


In [340]:
d[d['label'] == 1][['text','label']].sample(5)

,text,label
107573,ютуб видео александр солоник,1
61779,незнайка с волшебной палочкой смотреть онлайн,1
100114,фильм водный мир,1
54281,лучшие санатории россии рейтинг,1
73433,портретный очерк сочинение про друга,1


In [341]:
d[d['label'] == 3][['text','label']].sample(5)

,text,label
62664,новости от сб по вкладам,3
6704,new netent casinos,3
90247,собес харьков коминтерновский район,3
115492,lenovo g- обзор,3
84676,септик астра отрицательные отзывы,3


####   Я довольно долго пыталась подобрать наиболее разные запросы. План не удался( наверное стоит перед обучением модели оставить такие односложные запросы для последующего центрирования.


####   Ну а что если попробовать найти сколько действительно разных групп есть в сете?

In [37]:
a=[]
for i in range(len(d)):
    a.append(modeld.docvecs[i])

summ=list(range(3,29))#29 потому, что надоело ждать
for n in range(3,29):
    kmeans = KMeans(n_clusters=n,init='k-means++', random_state=0).fit(a)
    d['lab'] = kmeans.labels_
    summ[n-3] = kmeans.inertia_
summ

[164861.93977694336,
 161930.53460052723,
 159415.5556494498,
 157388.87923943636,
 155775.84076542515,
 154296.44933774622,
 152952.1080301792,
 151620.45288893612,
 150483.23744775436,
 149377.1442545151,
 148349.50294271475,
 147375.5287112365,
 146378.14154611283,
 145470.47054473317,
 144411.73588856874,
 143835.69401583413,
 143164.6716793776,
 142149.7420731538,
 141456.12323092218,
 140793.46457679465,
 140196.48305160124,
 139515.91677980358,
 139101.144745106,
 138556.25684689643,
 137978.5797269005,
 137242.24938460416,
 136846.80748002615,
 136123.43486119827,
 135504.93599178083]

In [323]:
pyplot.plot(range(3,32))

#### Поиграв с масштабом граффика, мне показалось, что наибольший изгиб графика в точке 4.
#### Посмотрим какие запросы ближе всего окажутся к центрам при таком подходе.

In [342]:
n=4
kmeans = KMeans(n_clusters=n, init='k-means++').fit(a)
for i in range(n):
    print(d.loc[modeld.docvecs.most_similar(positive=[kmeans.cluster_centers_[i]])[0][0],'text'])

джеки сэндлер фильмы
war of change
какое сочинение можно написать про кухню  класс
aragorn the king of magic land


#### Похоже это и есть самые далекие точки. Значит результат такой же плохой. Это может значить о том, что надо лучше чистить сет перед обучением. Скорее всего там слишком много шума, что мешает правильному определению смысла.

### В итоге лучше всего для выявления похожих себя показал Doc2Vec. однако он же совем не справился с класстеризацией. но скорее всего тут просто нужно уделить больше времени и на сам алгоритм .

# вторая задачка

#### 2.1  Я буду пользоваться простейшим вхождением описанным выше.

In [88]:
data = d.sample(10000)
related=[]
for i in data.index:
    try:
        str1 = set([x for x in data.loc[i,'bgrams'] if len(x)>2])
        for j in data.index[i+1:]:
            str2 = data.loc[j,'bgrams']
            cnt = 0
            for word in set([x for x in str1 if len(x)>2]):
                if word in str2:
                    cnt += 1
            if len(str2)>0 and cnt/len(str2) > 0.6:
                related.append((i,j))
                data.drop(j,inplace = True)
    except Exception:
        pass
rel = pd.DataFrame(related, columns=['query', 'related'])
rel['related_text'] =rel['related'].apply(lambda x: d.loc[x,'text'])
rel.head()

,query,related,related_text
0,4419,4823,онлайн поиск грузов
1,4341,3948,переводчик онлайн google
2,4341,9951,переводчик
3,5384,4758,рулетка
4,7003,10638,манка для ребенка год


#### 2.2

In [89]:
rel = pd.DataFrame(related, columns=['query', 'related'])
rel['related_text'] =rel['related'].apply(lambda x: d.loc[x,'text'])
r = rel.groupby('query').size().reset_index()
r.columns = ['query','qty']
rel = rel.merge(r, on='query')
rel.groupby(['qty','query', 'related_text']).size().sort_index(level = 'qty',ascending= False)

qty  query  related_text                                       
11   1148   эриус инструкция по применению                         1
            симвастатин инструкция по применению                   1
            ранитидин инструкция по применению                     1
            омепразол инструкция по применению                     2
            креон  инструкция по применению                        1
            интерферон инструкция по применению                    1
            деприм инструкция по применению                        1
            де-нол инструкция по применению                        1
            викасол инструкция по применению                       1
            вальсакор инструкция по применению                     1
8    1661   slot machine freeware                                  1
            slot games uk                                          1
            more free slot games                                   1
            live slot games            

In [90]:
последний столбец показывает колличество данного запроса в группе. 

#### 2.3 И опять группировка по похожим. Если брать Doc2Vec то можно найти среднюю сумму произведений векторов запросов каждого пользователя.
#### Либо также по вхождениям. Если есть лог кликов по ссылкам, можно также соединить и их множества.
